In [ ]:
from etl.data_quality_check import run_quality

run_quality()

2025-04-11 16:17:48,784 - INFO - [testcopy2.py:808] - Starting Data Quality Check Script for ALL ZIPS in USA
2025-04-11 16:17:48,785 - INFO - [testcopy2.py:821] - Scanning for zip code directories in: raw_data\USA
2025-04-11 16:17:48,786 - INFO - [testcopy2.py:828] - --- Processing Zip Code: 07008 ---
2025-04-11 16:17:48,788 - INFO - [testcopy2.py:63] - Scanning for JSON files in: raw_data\USA\07008\all_profiles
2025-04-11 16:18:05,580 - INFO - [testcopy2.py:73] - Scanned 3870 files. Found 1935 valid JSON files in raw_data\USA\07008\all_profiles.
2025-04-11 16:18:05,582 - INFO - [testcopy2.py:837] - Loaded 1935 profiles for zip 07008.
2025-04-11 16:18:05,583 - INFO - [testcopy2.py:63] - Scanning for JSON files in: raw_data\USA\07008\reviews\CHILD_CARE
2025-04-11 16:18:10,807 - INFO - [testcopy2.py:73] - Scanned 3904 files. Found 1952 valid JSON files in raw_data\USA\07008\reviews\CHILD_CARE.
2025-04-11 16:18:10,808 - INFO - [testcopy2.py:63] - Scanning for JSON files in: raw_data\USA\0


    Proposed Relational Data Model (Hybrid Flattening):
    ----------------------------------------------------
    NOTE: NestedServiceProfiles columns contain dynamically flattened fields
          from qualities, services, etc. prefixed by sub-profile type.
          JSON columns require further parsing/normalization in ETL/DB.

    1. Profiles (Main Caregiver Information)
       - profile_id (PK, VARCHAR) - From member.id
       - source_filename (VARCHAR)
       - member_firstName (VARCHAR, Nullable)
       - member_lastName (VARCHAR, Nullable)
       - member_gender (VARCHAR, Nullable)
       - member_displayName (VARCHAR, Nullable)
       - member_email (VARCHAR, Nullable)
       - member_primaryService (VARCHAR, Nullable)
       - member_hiResImageURL (VARCHAR, Nullable)
       - member_imageURL (VARCHAR, Nullable)
       - member_address_city (VARCHAR, Nullable)
       - member_address_state (VARCHAR, Nullable)
       - member_address_zip (VARCHAR, Nullable)
       - member_l

In [ ]:
from etl.sql_modeling import run_etl
run_etl()

2025-04-23 15:01:03,403 - INFO - [sql_modeling.py:76] - DB engine created.
2025-04-23 15:01:03,575 - INFO - [sql_modeling.py:1430] - DB session created.
2025-04-23 15:01:03,577 - INFO - [sql_modeling.py:1432] - Creating DB tables if needed...
2025-04-23 15:01:03,980 - INFO - [sql_modeling.py:1434] - Table check complete.
2025-04-23 15:01:03,982 - INFO - [sql_modeling.py:1438] - Reading main profiles: preprocessed_data\USA\main_profiles_processed.csv


In [1]:
from test import run_etl
run_etl()

2025-04-23 20:25:18,012 - INFO - [test.py:69] - Database engine created successfully.
2025-04-23 20:25:18,169 - INFO - [test.py:1399] - DB session created.
2025-04-23 20:25:18,171 - INFO - [test.py:1401] - Creating DB tables if needed...
2025-04-23 20:25:18,891 - INFO - [test.py:1403] - Table check complete.
2025-04-23 20:25:18,894 - INFO - [test.py:1407] - Reading main profiles: preprocessed_data\USA\main_profiles_processed.csv
2025-04-23 20:25:21,202 - INFO - [test.py:840] - Starting to load 11893 main profile records...
2025-04-23 20:29:22,368 - INFO - [test.py:1030] - Profiles load done. Loaded: 11893, Skipped: 0
2025-04-23 20:29:22,392 - INFO - [test.py:1411] - Committed main profiles.
2025-04-23 20:29:22,396 - INFO - [test.py:1419] - Found 6 nested profile CSV files.
2025-04-23 20:29:22,397 - INFO - [test.py:1426] - Reading nested profiles: preprocessed_data\USA\nested_childCareCaregiverProfile_processed.csv (Type: childCareCaregiverProfile)
2025-04-23 20:29:22,804 - INFO - [test

In [27]:
import json
from typing import Any, Optional
import pandas as pd

def parse_json_field(json_string: Optional[str], default_val: Any = None, dumps: bool = False) -> Any:
    """Safely parse a JSON string from CSV, return default on error or if None/empty."""
    if (
        pd.isna(json_string)
        or not isinstance(json_string, str)
        or not json_string.strip()
    ):
        return default_val
    try:
        if dumps:
            json_string = json_string.replace("'", '"')
            json_string =  json.dumps(json_string)  # Check if it's a valid JSON string
        return json.loads(json_string)
    except json.JSONDecodeError:
        return default_val

df = pd.read_csv('./preprocessed_data/USA/nested_childCareCaregiverProfile_processed.csv')
#df.iloc[1].get("childCareCaregiverProfile_payRange_hourlyRateFrom") #["childCareCaregiverProfile_payRange_hourlyRateFrom"]
print(parse_json_field(df.iloc[1].get("childCareCaregiverProfile_payRange_hourlyRateFrom"), dumps=True))
x = json.loads(parse_json_field(df.iloc[1].get("childCareCaregiverProfile_payRange_hourlyRateFrom"), dumps=True))
print(x["amount"])
#print(json.loads(df.iloc[1].get("childCareCaregiverProfile_payRange_hourlyRateFrom")))
data = json.loads('{"amount": "15", "currencyCode": "USD", "__typename": "Money"}')
#data["amount"]

{"amount": "15", "currencyCode": "USD", "__typename": "Money"}
15


In [2]:
import json

# Example JSON string
json_string = '''
{
  "name": "Alice",
  "is_active": true,
  "count": 42,
  "status": "ok",
  "is_verified": false,
  "maybe_boolean": "true"
}
'''

# Parse the JSON string into a Python dictionary
data = json.loads(json_string)

# --- Checking specific values ---

# Check 'is_active' which is a JSON true -> Python True
value_active = data.get('is_active')
is_bool_active = isinstance(value_active, bool)
print(f"Value of 'is_active': {value_active} (Type: {type(value_active)})")
print(f"Is 'is_active' a boolean? {is_bool_active}") # Output: True

print("-" * 20)

# Check 'is_verified' which is a JSON false -> Python False
value_verified = data.get('is_verified')
is_bool_verified = isinstance(value_verified, bool)
print(f"Value of 'is_verified': {value_verified} (Type: {type(value_verified)})")
print(f"Is 'is_verified' a boolean? {is_bool_verified}") # Output: True

print("-" * 20)

# Check 'count' which is a number -> Python int
value_count = data.get('count')
is_bool_count = isinstance(value_count, bool)
print(f"Value of 'count': {value_count} (Type: {type(value_count)})")
print(f"Is 'count' a boolean? {is_bool_count}") # Output: False

print("-" * 20)

# Check 'maybe_boolean' which is a string -> Python str
value_maybe = data.get('maybe_boolean')
is_bool_maybe = isinstance(value_maybe, bool)
print(f"Value of 'maybe_boolean': {value_maybe} (Type: {type(value_maybe)})")
print(f"Is 'maybe_boolean' a boolean? {is_bool_maybe}") # Output: False

print("-" * 20)

# --- Iterating and checking ---
print("Iterating through all values:")
for key, value in data.items():
    if isinstance(value, bool):
        print(f"  Key '{key}' has a boolean value: {value}")
    else:
        print(f"  Key '{key}' has a non-boolean value: {value} (Type: {type(value).__name__})")

Value of 'is_active': True (Type: <class 'bool'>)
Is 'is_active' a boolean? True
--------------------
Value of 'is_verified': False (Type: <class 'bool'>)
Is 'is_verified' a boolean? True
--------------------
Value of 'count': 42 (Type: <class 'int'>)
Is 'count' a boolean? False
--------------------
Value of 'maybe_boolean': true (Type: <class 'str'>)
Is 'maybe_boolean' a boolean? False
--------------------
Iterating through all values:
  Key 'name' has a non-boolean value: Alice (Type: str)
  Key 'is_active' has a boolean value: True
  Key 'count' has a non-boolean value: 42 (Type: int)
  Key 'status' has a non-boolean value: ok (Type: str)
  Key 'is_verified' has a boolean value: False
  Key 'maybe_boolean' has a non-boolean value: true (Type: str)
